<a href="https://colab.research.google.com/github/kashfay110/stock-market-python/blob/main/svr_ann_collab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install mplfinance

     |████████████████████████████████| 71kB 3.5MB/s 


In [ ]:
pip install -U scikit-learn

     |████████████████████████████████| 22.3MB 34.6MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [ ]:
# import quandl
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import mplfinance as mpf
import keras
from keras.models import Sequential
from keras.layers import Dense
import os

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
sectorname = 'Real Estate'
stockname = ['CCI','DLR','EQIX','PLD','PSA','SBAC','SPG','WELL']

In [ ]:
for s in stockname:
  pathname = (f'/content/gdrive/My Drive/Colab Notebooks/dataset/Sectors/{sectorname}/{s}.csv')
  df = pd.read_csv (pathname)
  filename = os.path.basename(pathname)
  filename_grab = os.path.splitext(filename)[0]

  df = df[['Adj Close']]

  # A variable for predicting 'forecast_out' days out in future
  for i in [1,2,3,4,5,10,15,30]:
    forecast_out = i
    # Create another column (dependent variable) shifted 'forecast_out' units up
    df['Prediction'] = df[["Adj Close"]].shift(-forecast_out)

    print(s,i)
    # Create the independent dataset
    # Convert the dataframe to numpy array
    X = np.array(df.drop(['Prediction'],1))
    # Remove the last 'forecast_out' rows
    X = X[:-forecast_out]
    #print(X)
    y = np.array(df['Prediction'])
    # Get all of the y values except the last 'forecast_out' rows
    y = y[:-forecast_out] 

    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    x_forecast = np.array(df.drop(['Prediction'],1))[-forecast_out:]

    ann_relu = Sequential()
    ann_relu.add(Dense(200, input_dim=1, activation='relu'))
    ann_relu.add(Dense(200, input_dim=200, activation='relu'))
    ann_relu.add(Dense(200, input_dim=200, activation='relu'))
    ann_relu.add(Dense(1, activation='linear'))

    keras.optimizers.Adam(lr=0.1, beta_1=0.9, beta_2=0.999, amsgrad=False)
    ann_relu.compile(loss='mean_squared_error', optimizer='RMSprop', metrics=['mean_absolute_percentage_error'])

    ann_relu.summary()

    history = ann_relu.fit(x_train, y_train, epochs=200, batch_size=32,validation_split=0.15,verbose=1)

    y_pred_ann = ann_relu.predict(x_test)

    mse_ann = mean_squared_error(y_test, y_pred_ann)
    print(f'The MSE for the ANN algorithm is: {mse_ann}')

    rmse_ann = mean_squared_error(y_test, y_pred_ann, squared=False)
    print(f'The RMSE for the ANN algorithm is: {rmse_ann}')

    mae_ann = mean_absolute_error(y_test, y_pred_ann)
    print(f'The MAE for the ANN algorithm is: {mae_ann}')

    mape_ann = mean_absolute_percentage_error(y_test, y_pred_ann)
    print(f'The MAPE for the ANN algorithm is: {mape_ann}')

    n_array = np.array([])
    new_array = np.append(n_array, [mse_ann, rmse_ann, mae_ann, mape_ann])
    print(new_array)

    new_df = pd.DataFrame(new_array)

    filepath = f'/content/gdrive/My Drive/Colab Notebooks/results/Sectors/{sectorname}/{forecast_out}Days/{filename_grab}_ann_results.xlsx'
    new_df.to_excel(filepath, index=False)

    keras.backend.clear_session()


CCI 1


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 200)               400       
_________________________________________________________________
dense_1 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_2 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 201       
Total params: 81,001
Trainable params: 81,001
Non-trainable params: 0
_________________________________________________________________
Epoch 1/200
61/61 [==============================] - 16s 14ms/step - loss: 992.5146 - mean_absolute_percentage_error: 17.4632 - val_loss: 63.4960 - val_mean_absolute_percentage_error: 8.7102
Epoch 2/200
61/61 [==============================] - 0s 3ms

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


61/61 [==============================] - 2s 8ms/step - loss: 924.9721 - mean_absolute_percentage_error: 16.3084 - val_loss: 97.8630 - val_mean_absolute_percentage_error: 11.0334
Epoch 2/200
61/61 [==============================] - 0s 3ms/step - loss: 50.8744 - mean_absolute_percentage_error: 5.8364 - val_loss: 12.2134 - val_mean_absolute_percentage_error: 3.1786
Epoch 3/200
61/61 [==============================] - 0s 3ms/step - loss: 52.8701 - mean_absolute_percentage_error: 5.9536 - val_loss: 7.2142 - val_mean_absolute_percentage_error: 2.0458
Epoch 4/200
61/61 [==============================] - 0s 3ms/step - loss: 33.9126 - mean_absolute_percentage_error: 5.6147 - val_loss: 16.7029 - val_mean_absolute_percentage_error: 4.0198
Epoch 5/200
61/61 [==============================] - 0s 3ms/step - loss: 31.2447 - mean_absolute_percentage_error: 4.4606 - val_loss: 5.1696 - val_mean_absolute_percentage_error: 1.5235
Epoch 6/200
61/61 [==============================] - 0s 3ms/step - loss: 20.

...

61/61 [==============================] - 2s 12ms/step - loss: 262.7804 - mean_absolute_percentage_error: 17.3840 - val_loss: 9.9774 - val_mean_absolute_percentage_error: 3.8576
Epoch 2/200
61/61 [==============================] - 0s 4ms/step - loss: 25.0701 - mean_absolute_percentage_error: 7.4945 - val_loss: 13.7162 - val_mean_absolute_percentage_error: 4.9609
Epoch 3/200
61/61 [==============================] - 0s 4ms/step - loss: 21.4795 - mean_absolute_percentage_error: 7.1053 - val_loss: 10.3337 - val_mean_absolute_percentage_error: 3.9074
Epoch 4/200
61/61 [==============================] - 0s 4ms/step - loss: 16.8682 - mean_absolute_percentage_error: 6.0800 - val_loss: 17.2061 - val_mean_absolute_percentage_error: 6.4686
Epoch 5/200
61/61 [==============================] - 0s 5ms/step - loss: 16.0481 - mean_absolute_percentage_error: 5.8703 - val_loss: 10.0990 - val_mean_absolute_percentage_error: 3.9859
Epoch 6/200
61/61 [==============================] - 0s 4ms/step - loss: 16

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


61/61 [==============================] - 2s 13ms/step - loss: 440.2067 - mean_absolute_percentage_error: 23.4779 - val_loss: 29.2758 - val_mean_absolute_percentage_error: 5.5465
Epoch 2/200
61/61 [==============================] - 0s 4ms/step - loss: 25.3919 - mean_absolute_percentage_error: 7.3153 - val_loss: 36.9570 - val_mean_absolute_percentage_error: 8.7307
Epoch 3/200
61/61 [==============================] - 0s 4ms/step - loss: 26.1366 - mean_absolute_percentage_error: 7.1775 - val_loss: 26.9588 - val_mean_absolute_percentage_error: 5.3179
Epoch 4/200
61/61 [==============================] - 0s 5ms/step - loss: 22.3906 - mean_absolute_percentage_error: 6.9751 - val_loss: 66.1071 - val_mean_absolute_percentage_error: 13.8954
Epoch 5/200
61/61 [==============================] - 0s 5ms/step - loss: 22.3128 - mean_absolute_percentage_error: 6.9205 - val_loss: 32.3108 - val_mean_absolute_percentage_error: 7.5577
Epoch 6/200
61/61 [==============================] - 0s 5ms/step - loss: 

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


61/61 [==============================] - 2s 12ms/step - loss: 515.5340 - mean_absolute_percentage_error: 27.9112 - val_loss: 27.2685 - val_mean_absolute_percentage_error: 6.4209
Epoch 2/200
61/61 [==============================] - 0s 4ms/step - loss: 39.6664 - mean_absolute_percentage_error: 8.6410 - val_loss: 27.4200 - val_mean_absolute_percentage_error: 6.9903
Epoch 3/200
61/61 [==============================] - 0s 5ms/step - loss: 42.5895 - mean_absolute_percentage_error: 8.5666 - val_loss: 63.4896 - val_mean_absolute_percentage_error: 12.5925
Epoch 4/200
61/61 [==============================] - 0s 4ms/step - loss: 41.8639 - mean_absolute_percentage_error: 8.4379 - val_loss: 37.4313 - val_mean_absolute_percentage_error: 9.1123
Epoch 5/200
61/61 [==============================] - 0s 5ms/step - loss: 37.6309 - mean_absolute_percentage_error: 8.1988 - val_loss: 28.7717 - val_mean_absolute_percentage_error: 6.7420
Epoch 6/200
61/61 [==============================] - 0s 5ms/step - loss: 

In [ ]:
# import pandas as pd
# xyz = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset/Sectors/Utilities/AWK.csv') 
# print(xyz.head(1))

In [ ]:
# # Get the stock data
# #df = quandl.get("WIKI/FB")
# #df = pd.read_csv (r'dataset/FB.csv')
# # Take a look at data
# #print(df.head())

# pathname = r'AML.L.csv'

# from google.colab import files
# uploaded = files.upload()
# import io
# df = pd.read_csv(io.BytesIO(uploaded[pathname]))
# df.head()

# #print(df2.head())
# # Get the Adjusted Close
# df = df[['Adj Close']]
# # Take a look at new data
# #print(df.head())

In [ ]:
# filename = os.path.basename(pathname)
# filename_grab = os.path.splitext(filename)[0]

In [ ]:
# # A variable for predicting 'forecast_out' days out in future
# forecast_out = 5

In [ ]:
# # Create another column (dependent variable) shifted 'forecast_out' units up
# df['Prediction'] = df[["Adj Close"]].shift(-forecast_out)
# # print(df.tail())

In [ ]:
# # Create the independent dataset
# # Convert the dataframe to numpy array
# X = np.array(df.drop(['Prediction'],1))
# # Remove the last 'forecast_out' rows
# X = X[:-forecast_out]
# #print(X)

In [ ]:
# # Create the dependent dataset
# # Convert the dataframe to numpy array (All of the values including the NaNs)
# y = np.array(df['Prediction'])
# # Get all of the y values except the last 'forecast_out' rows
# y = y[:-forecast_out]
# #print(y)

In [ ]:
# # Split the data into 80% training and 20% testing
# x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# # print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
# # train_set = x_train + y_train
# # test_set = x_test + y_test

In [ ]:
# #Set x_forecast equal to the last forecast_out rows of the original dat set from Adj. Close column
# x_forecast = np.array(df.drop(['Prediction'],1))[-forecast_out:]
# #print(x_forecast)

In [ ]:
# ann_relu = Sequential()
# ann_relu.add(Dense(200, input_dim=1, activation='relu'))
# ann_relu.add(Dense(200, input_dim=200, activation='relu'))
# ann_relu.add(Dense(200, input_dim=200, activation='relu'))
# ann_relu.add(Dense(1, activation='linear'))

In [ ]:
# keras.optimizers.Adam(lr=0.1, beta_1=0.9, beta_2=0.999, amsgrad=False)
# ann_relu.compile(loss='mean_squared_error', optimizer='RMSprop', metrics=['mean_absolute_percentage_error'])

In [ ]:
# ann_relu.summary()

In [ ]:
# history = ann_relu.fit(x_train, y_train, epochs=200, batch_size=32,validation_split=0.15,verbose=1)

In [ ]:
# y_pred_ann = ann_relu.predict(x_test)

In [ ]:
# mse_ann = mean_squared_error(y_test, y_pred_ann)
# print(f'The MSE for the ANN algorithm is: {mse_ann}')

In [ ]:
# rmse_ann = mean_squared_error(y_test, y_pred_ann, squared=False)
# print(f'The RMSE for the ANN algorithm is: {rmse_ann}')

In [ ]:
# mae_ann = mean_absolute_error(y_test, y_pred_ann)
# print(f'The MAE for the ANN algorithm is: {mae_ann}')

In [ ]:
# mape_ann = mean_absolute_percentage_error(y_test, y_pred_ann)
# print(f'The MAPE for the ANN algorithm is: {mape_ann}')

In [ ]:
# n_array = np.array([])
# new_array = np.append(n_array, [mse_ann, rmse_ann, mae_ann, mape_ann])
# print(new_array)

In [ ]:
# new_df = pd.DataFrame(new_array)

In [ ]:
# filepath = f'{filename_grab}_ann_results.xlsx'
# new_df.to_excel(filepath, index=False)

In [ ]:
# keras.backend.clear_session()